# Day 1B: Advanced Agent Architectures

## 🎯 Learning Objectives

By the end of this notebook, you will:
- Understand why single agents can be limiting for complex tasks
- Learn how to build multi-agent systems with specialized roles
- Master different workflow patterns: Sequential, Parallel, and Loop
- Know when to use each architecture pattern

## 📚 Prerequisites

- Completion of **Day 1A: Introduction to Agent Architectures**
- Understanding of basic agent components (Agent, Tools, Runner)
- Google API key (stored in `.env` file)

---

## Quick Recap from Day 1A

In Day 1A, we created a **single agent** that could:
- Understand user questions
- Use Google Search for current information
- Provide helpful responses

### The Limitation of Single Agents

While single agents are powerful, they face challenges with complex tasks:

```
       ┌─────────────────────────────────────┐
       │         Single "Do-It-All" Agent    │
       │                                     │
       │  ❌ Long, confusing instructions    │
       │  ❌ Hard to debug (what failed?)    │
       │  ❌ Difficult to maintain           │
       │  ❌ Unreliable for complex tasks    │
       └─────────────────────────────────────┘
```

### The Solution: Agent Teams

Instead of one "do-it-all" agent, we'll build **teams of specialists**:

```
              ┌─────────────┐  ┌─────────────┐  ┌─────────────┐
              │   Research  │  │   Writer    │  │   Editor    │
              │   Agent     │  │   Agent     │  │   Agent     │
              │             │  │             │  │             │
              │    Find     │  │   Create    │  │   Polish    │
              │    info     │  │   content   │  │   content   │
              └─────────────┘  └─────────────┘  └─────────────┘
                     │                │                │
                     └────────────────┼────────────────┘
                                   │
                            ┌───────────────┐
                            │ Coordinator   │
                            │   Agent       │
                            │               │
                            │   Orchestrate │
                            │    workflow   │
                            └───────────────┘
```

Each agent has **one clear job**, making them:
- ✅ Easier to build and test
- ✅ More reliable and focused
- ✅ Simpler to debug and maintain

---

## Setting Up Our Environment

Let's install the required packages (same as Day 1A):

In [ ]:
import sys
import subprocess
import importlib

def install_package(package):
    """Install a Python package using pip"""
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])

# Check if python-dotenv is installed, install if not
try:
    import dotenv
    print("✅ python-dotenv is already installed")
except ImportError:
    print("Installing python-dotenv...")
    install_package("python-dotenv")
    import dotenv
    print("✅ python-dotenv installed successfully")

# Check if google-adk is installed, install if not
try:
    import google.adk.agents
    print("✅ google-adk is already installed")
except ImportError:
    print("Installing google-adk...")
    install_package("google-adk")
    import google.adk.agents
    print("✅ google-adk installed successfully")

# Check if google-genai is installed, install if not
try:
    import google.genai
    print("✅ google-genai is already installed")
except ImportError:
    print("Installing google-genai...")
    install_package("google-genai")
    import google.genai
    print("✅ google-genai installed successfully")

print("\n🎉 All required packages are installed and ready!")

## Setting Up Authentication

Load the API key from your `.env` file:

In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

try:
    # Get API key from environment variable (from .env file)
    GOOGLE_API_KEY = os.environ.get("GOOGLE_API_KEY")
    
    if not GOOGLE_API_KEY:
        raise ValueError("GOOGLE_API_KEY not found in environment variables")
    
    # Set the API key for ADK
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "FALSE"
    
    print("✅ Gemini API key setup complete.")
    
except Exception as e:
    print(f"🔑 Authentication Error: Please make sure you have set 'GOOGLE_API_KEY' in your .env file. Details: {e}")

## Importing ADK Components

For advanced architectures, we need additional components:

In [ ]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types
import asyncio

print("✅ ADK components imported successfully.")
print("📦 Available components:")
print("  • Agent - Basic agent class")
print("  • SequentialAgent - Runs agents in order")
print("  • ParallelAgent - Runs agents simultaneously")
print("  • LoopAgent - Runs agents repeatedly")
print("  • AgentTool - Uses agents as tools")
print("  • FunctionTool - Uses Python functions as tools")

---

## Section 1: Multi-Agent Systems

Let's build our first multi-agent system with two specialized agents:

### Architecture Overview

```
        ┌─────────────────────────────────────────────────────────┐
        │                 Multi-Agent System                      │
        │                                                         │
        │  User Question ──► Coordinator Agent                    │
        │                    │                                    │
        │          ┌─────────┴─────────┐                          │
        │          │                   │                          │
        │          ▼                   ▼                          │
        │  ┌─────────────┐      ┌─────────────┐                   │
        │  │   Research  │      │ Summarizer  │                   │
        │  │   Agent     │      │   Agent     │                   │
        │  │             │      │             │                   │
        │  │   Find      │      │   Create    │                   │
        │  │   info      │      │   summary   │                   │
        │  └─────────────┘      └─────────────┘                   │
        │          │                   │                          │
        │          └─────────┬─────────┘                          │
        │                    │                                    │
        │                    ▼.                                   │
        │              Final Summary                              │
        └─────────────────────────────────────────────────────────┘
```

### Agent Roles

1. **Research Agent** - Searches for information using Google Search
2. **Summarizer Agent** - Creates concise summaries from research findings
3. **Coordinator Agent** - Orchestrates the workflow by calling the other agents

### Creating the Research Agent

In [ ]:
# Research Agent: Its job is to use the google_search tool and present findings.
research_agent = Agent(
    name="ResearchAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic and present the findings with citations.
    
    Important:
    - Focus on finding recent, reliable information
    - Include sources/citations when possible
    - Keep your findings concise but comprehensive
    - Do not summarize - just present the research findings""",
    tools=[google_search],
    output_key="research_findings", # The result will be stored with this key
)

print("✅ research_agent created.")
print(f"🔍 Research Agent will store findings in: '{research_agent.output_key}'")

### Creating the Summarizer Agent

In [ ]:
# Summarizer Agent: Its job is to summarize the text it receives.
summarizer_agent = Agent(
    name="SummarizerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Read the provided research findings: {research_findings}

Create a concise summary as a bulleted list with 3-5 key points.

Guidelines:
- Extract the most important information
- Use clear, concise bullet points
- Maintain accuracy to the source material
- Focus on actionable insights""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")
print(f"📝 Summarizer Agent will store summary in: '{summarizer_agent.output_key}'")
print(f"🔗 It reads from: '{{research_findings}}' (from Research Agent)")

### Creating the Coordinator Agent

In [ ]:
# Root Coordinator: Orchestrates the workflow by calling the sub-agents as tools.
root_agent = Agent(
    name="ResearchCoordinator",
    model="gemini-2.5-flash-lite",
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.

Follow these steps EXACTLY in order:
1. First, you MUST call the `ResearchAgent` tool to find relevant information on the topic provided by the user.
2. Next, after receiving the research findings, you MUST call the `SummarizerAgent` tool to create a concise summary.
3. Finally, present the final summary clearly to the user as your response.

Important:
- Do not skip any steps
- Wait for each agent to complete before calling the next
- Present the final summary as your answer to the user""",
    # We wrap the sub-agents in `AgentTool` to make them callable tools for the root agent.
    tools=[
        AgentTool(research_agent),
        AgentTool(summarizer_agent)
    ],
)

print("✅ root_agent created.")
print(f"🎯 Coordinator has {len(root_agent.tools)} tools available:")
for tool in root_agent.tools:
    print(f"  • {tool.agent.name}")

### Testing the Multi-Agent System

In [ ]:
# Test the multi-agent architecture
runner = InMemoryRunner(agent=root_agent)

print("🚀 Testing multi-agent system...")
print("📝 Question: What are the latest advancements managing PII data for a Snowflake Data Warehouse?")
print("\n⏳ Processing...")

response = await runner.run_debug("What are the latest advancements managing PII data for a Snowflake Data Warehouse?")

print("\n🤖 Final Response:")
print(response)

### How It Works: Behind the Scenes

You might be wondering: How does the research agent know what to research? How does information flow between agents?

#### Information Flow Explained

```
1. User asks: "What are the latest advancements managing PII data for Snowflake?"
            │
            ▼
2. Coordinator Agent receives the question
            │
            ▼
3. Coordinator calls ResearchAgent tool
             │
             ▼
4. ADK automatically passes the original question to ResearchAgent
            │
            ▼
5. ResearchAgent uses google_search and stores results in 'research_findings'
            │
            ▼
6. Coordinator calls SummarizerAgent tool
            │
            ▼
7. SummarizerAgent reads from {research_findings} and creates summary
            │
            ▼
8. Final summary is presented to user
```

#### Key Concepts

- **Automatic Question Passing**: When Coordinator calls `AgentTool(research_agent)`, ADK automatically passes the original user question
- **State Management**: Research outputs to `research_findings` key, Summarizer reads from `{research_findings}`
- **Tool Wrapping**: `AgentTool()` makes agents callable as tools
- **Clear Separation**: Each agent has one specific job

---

## Section 2: Sequential Workflows

The previous multi-agent system relied on detailed instructions to force the LLM to run steps in order. This can be unreliable. A better approach for ordered tasks is **SequentialAgent**.

### When to Use Sequential Workflows

✅ **Use Sequential when:**
- Order matters (each step builds on the previous)
- You need a guaranteed, fixed pipeline
- Tasks are linear and dependent

❌ **Don't use Sequential when:**
- Tasks can run independently
- Order doesn't matter
- You need dynamic decision-making

### Sequential Architecture

```
        ┌─────────────────────────────────────────────────────────┐
        │                 Sequential Workflow                     │
        │                                                         │
        │  Input ──► Agent 1 ──► Agent 2 ──► Agent 3 ──► Output   │
        │             │           │           │                   │
        │             ▼           ▼           ▼                   │
        │         Outline      Writer      Editor                 │
        │         Agent        Agent       Agent                  │
        │                                                         │
        │  Each agent's output becomes the next agent's input     │
        └─────────────────────────────────────────────────────────┘
```

### Example: Blog Post Pipeline

Let's create a 3-step pipeline for creating blog posts:

1. **Outline Agent** - Creates a blog outline
2. **Writer Agent** - Writes the blog post
3. **Editor Agent** - Edits and polishes the post

### Creating the Outline Agent

In [ ]:
# Outline Agent: Creates the initial blog post outline.
outline_agent = Agent(
    name="OutlineAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought
    
Make the outline engaging and well-structured.""",
    output_key="blog_outline",
)

print("✅ outline_agent created.")
print(f"📋 Outline Agent will store outline in: '{outline_agent.output_key}'")

### Creating the Writer Agent

In [ ]:
# Writer Agent: Writes the full blog post based on the outline.
writer_agent = Agent(
    name="WriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Following this outline strictly: {blog_outline}

Write a brief, 200 to 300-word blog post with an engaging and informative tone.

Guidelines:
- Follow the outline structure exactly
- Keep within the word count
- Use clear, accessible language
- Include a compelling introduction and conclusion""",
    output_key="blog_draft",
)

print("✅ writer_agent created.")
print(f"✍️ Writer Agent will store draft in: '{writer_agent.output_key}'")
print(f"🔗 It reads from: '{{blog_outline}}' (from Outline Agent)")

### Creating the Editor Agent

In [ ]:
# Editor Agent: Edits and polishes the draft from the writer agent.
editor_agent = Agent(
    name="EditorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Edit this draft: {blog_draft}

Your task is to polish the text by:
- Fixing any grammatical errors
- Improving the flow and sentence structure
- Enhancing overall clarity
- Ensuring consistent tone
- Maintaining the original meaning and word count""",
    output_key="final_blog",
)

print("✅ editor_agent created.")
print(f"✅ Editor Agent will store final blog in: '{editor_agent.output_key}'")
print(f"🔗 It reads from: '{{blog_draft}}' (from Writer Agent)")

### Creating the Sequential Pipeline

In [ ]:
# Create a sequential agent that runs the agents in order
blog_pipeline = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")
print(f"🔄 Pipeline will execute {len(blog_pipeline.sub_agents)} agents in order:")
for i, agent in enumerate(blog_pipeline.sub_agents, 1):
    print(f"  {i}. {agent.name}")

### Testing the Sequential Workflow

In [ ]:
# Test the sequential workflow
blog_runner = InMemoryRunner(agent=blog_pipeline)

print("🚀 Testing sequential workflow...")
print("📝 Topic: Write a blog post about the benefits of masking policies for a data warehouse dealing with PII data")
print("\n⏳ Processing through pipeline...")

blog_response = await blog_runner.run_debug("Write a blog post about the benefits of masking policies for a data warehouse dealing with PII data")

print("\n📄 Final Blog Post:")
print(blog_response)

---

## Section 3: Parallel Workflows

Sequential workflows are great for dependent tasks, but what if you have independent tasks that could run simultaneously? That's where **ParallelAgent** comes in.

### When to Use Parallel Workflows

✅ **Use Parallel when:**
- Tasks are independent and don't depend on each other
- Speed is important
- You can execute tasks concurrently

❌ **Don't use Parallel when:**
- Tasks need to wait for each other's output
- Order matters
- Tasks share resources that can't be accessed simultaneously

### Parallel Architecture

```
        ┌───────────────────────────────────────────────────────────┐
        │                 Parallel Workflow                         │
        │                                                           │
        │  Input ┌─────────┐ ┌─────────┐ ┌─────────┐                │
        │   ───► │ Agent 1 │ │ Agent 2 │ │ Agent 3 │───► Aggregator │
        │        │         │ │         │ │         │                │
        │        │  Tech   │ │ Health  │ │Finance  │                │
        │        │Research │ │Research │ │Research │                │
        │        └─────────┘ └─────────┘ └─────────┘                │
        │              │           │           │                    │
        │              └───────────┼───────────┘                    │
        |                          │                                │
        |                          ▼                                │
        │                 Combined Results                          │
        └───────────────────────────────────────────────────────────┘
```

### Example: Multi-Domain Research Team

Let's create a system that researches different domains in parallel:

1. **Tech Researcher** - Researches AI/ML trends
2. **Health Researcher** - Researches medical breakthroughs
3. **Finance Researcher** - Researches fintech trends
4. **Aggregator Agent** - Combines all findings

### Creating the Research Agents

In [ ]:
# Tech Researcher: Focuses on AI and ML trends.
tech_researcher = Agent(
    name="TechResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research the latest AI/ML trends. Include:
    - 3 key developments
    - The main companies involved
    - The potential impact
    
Keep the report very concise (around 100 words).""",
    tools=[google_search],
    output_key="tech_research",
)

# Health Researcher: Focuses on medical breakthroughs.
health_researcher = Agent(
    name="HealthResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research recent medical breakthroughs. Include:
    - 3 significant advances
    - Their practical applications
    - Estimated timelines
    
Keep the report concise (around 100 words).""",
    tools=[google_search],
    output_key="health_research",
)

# Finance Researcher: Focuses on fintech trends.
finance_researcher = Agent(
    name="FinanceResearcher",
    model="gemini-2.5-flash-lite",
    instruction="""Research current fintech trends. Include:
    - 3 key trends
    - Their market implications
    - The future outlook
    
Keep the report concise (around 100 words).""",
    tools=[google_search],
    output_key="finance_research",
)

print("✅ All research agents created.")
print("🔍 Research team ready:")
for agent in [tech_researcher, health_researcher, finance_researcher]:
    print(f"  • {agent.name} → stores in '{agent.output_key}'")

### Creating the Aggregator Agent

In [ ]:
# The AggregatorAgent runs *after* the parallel step to synthesize the results.
aggregator_agent = Agent(
    name="AggregatorAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Combine these three research findings into a single executive summary:

**Technology Trends:**
{tech_research}
    
**Health Breakthroughs:**
{health_research}
    
**Finance Innovations:**
{finance_research}
    
Your summary should:
- Highlight common themes across domains
- Identify surprising connections
- Present the most important key takeaways
- Be around 200 words
- Provide actionable insights for decision-makers""",
    output_key="executive_summary",
)

print("✅ aggregator_agent created.")
print(f"📊 Aggregator will read from all research outputs and store in '{aggregator_agent.output_key}'")

### Creating the Parallel System

In [ ]:
# The ParallelAgent runs all its sub-agents simultaneously.
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

# This SequentialAgent defines the high-level workflow: 
# 1. Run the parallel team first, 
# 2. Then run the aggregator.
research_system = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")
print("🔄 System workflow:")
print("  1. Run 3 research agents in parallel")
print("  2. Aggregate results into executive summary")

### Testing the Parallel Workflow

In [ ]:
# Test the parallel workflow
research_runner = InMemoryRunner(agent=research_system)

print("🚀 Testing parallel workflow...")
print("📊 Task: Run the daily executive briefing on Tech, Health, and Finance")
print("\n⏳ Running research in parallel...")

research_response = await research_runner.run_debug("Run the daily executive briefing on Tech, Health, and Finance")

print("\n📈 Executive Summary:")
print(research_response)

---

## Section 4: Loop Workflows

Sometimes, one pass through a workflow isn't enough. For tasks that require refinement and improvement, we need **LoopAgent**.

### When to Use Loop Workflows

✅ **Use Loop when:**
- Iterative improvement is needed
- Quality refinement matters
- You need repeated cycles of feedback and revision

❌ **Don't use Loop when:**
- One pass is sufficient
- Tasks are straightforward
- You need guaranteed completion in fixed time

### Loop Architecture

```
    ┌──────────────────────────────────────────────────────────┐
    │                   Loop Workflow                          │
    │                                                          │
    │  Input ──► Initial Writer ──► ┌──────────────────────┐   │
    │                           │   │   Loop (max N times) │   │
    │                           │   │                      │   │
    │                           │   │  ┌─────────────┐     │   │
    │                           │   │  │   Critic    │     │   │
    │                           │   │  │   Agent     │     │   │
    │                           │   │  └──────┬──────┘     │   │
    │                           │   │         │            │   │
    │                           │   │         ▼            │   │
    │                           │   │  ┌─────────────┐     │   │
    │                           │   │  │  Refiner    │     │   │
    │                           │   │  │   Agent     │     │   │
    │                           │   │  └─────────────┘     │   │
    │                           │   │         │            │   │
    │                           │   │     (if not          │   │
    │                           │   │      approved)       │   │
    │                           │   └──────────────────────┘   │
    │                           │            │                 │
    │                           ▼            ▼                 │
    │                      Final Output (when approved)        │
    └──────────────────────────────────────────────────────────┘

```

### Example: Story Refinement Loop

Let's create a system that writes and refines a story until it's approved:

1. **Initial Writer Agent** - Writes the first draft
2. **Critic Agent** - Reviews and critiques the story
3. **Refiner Agent** - Improves the story based on feedback

### Creating the Initial Writer Agent

In [ ]:
# This agent runs ONCE at the beginning to create the first draft.
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model="gemini-2.5-flash-lite",
    instruction="""Based on the user's prompt, write the first draft of a short story (around 100-150 words).

Guidelines:
- Create an engaging beginning, middle, and end
- Include at least one character and a clear plot
- Output only the story text, with no introduction or explanation""",
    output_key="current_story",
)

print("✅ initial_writer_agent created.")
print(f"✍️ Initial writer will store story in: '{initial_writer_agent.output_key}'")

### Creating the Critic Agent

In [ ]:
# This agent's only job is to provide feedback or the approval signal.
critic_agent = Agent(
    name="CriticAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a constructive story critic. Review the story provided below.

Story: {current_story}
    
Evaluate the story's:
- Plot coherence and engagement
- Character development
- Pacing and flow
- Overall quality

Response format:
- If the story is well-written and complete, you MUST respond with the exact phrase: "APPROVED"
- Otherwise, provide 2-3 specific, actionable suggestions for improvement""",
    output_key="critique",
)

print("✅ critic_agent created.")
print(f"🔍 Critic will store feedback in: '{critic_agent.output_key}'")
print(f"🔗 It reads from: '{{current_story}}'")

### Creating the Exit Function and Refiner Agent

In [ ]:
# This is the function that the RefinerAgent will call to exit the loop.
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating the story is finished."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

print("✅ exit_loop function created.")

# This agent refines the story based on critique OR calls the exit_loop function.
refiner_agent = Agent(
    name="RefinerAgent",
    model="gemini-2.5-flash-lite",
    instruction="""You are a story refiner. You have a story draft and critique.
    
Story Draft: {current_story}
Critique: {critique}
    
Your task is to analyze the critique:
- IF the critique is EXACTLY "APPROVED", you MUST call the `exit_loop` function and nothing else.
- OTHERWISE, rewrite the story draft to fully incorporate the feedback from the critique.

When rewriting:
- Address all the critic's suggestions
- Maintain the original story's core elements
- Keep the word count similar (100-150 words)""",
    output_key="current_story", # Overwrites the story with the new version
    tools=[FunctionTool(exit_loop)],
)

print("✅ refiner_agent created.")
print(f"🔧 Refiner has {len(refiner_agent.tools)} tool available: exit_loop")

### Creating the Loop System

In [ ]:
# The LoopAgent contains the agents that will run repeatedly: Critic -> Refiner.
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=3, # Prevents infinite loops
)

# The root agent is a SequentialAgent that defines the overall workflow: Initial Write -> Refinement Loop.
story_pipeline = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")
print(f"🔄 Loop will run max {story_refinement_loop.max_iterations} iterations")
print("📝 Pipeline workflow:")
print("  1. Write initial story")
print("  2. Enter refinement loop (critic → refiner) until approved")

### Testing the Loop Workflow

In [ ]:
# Test the loop workflow
story_runner = InMemoryRunner(agent=story_pipeline)

print("🚀 Testing loop workflow...")
print("📖 Prompt: Write a short story about AI Engineers discovering a hidden code in ancient manuscripts")
print("\n⏳ Writing and refining story...")

story_response = await story_runner.run_debug("Write a short story about AI Engineers discovering a hidden code in ancient manuscripts")

print("\n📄 Final Story:")
print(story_response)

---
## 📚 Additional Resources

- [ADK Agent Patterns Guide](https://google.github.io/adk-docs/agent-patterns/)
- [Advanced Agent Architectures](https://google.github.io/adk-docs/advanced-architectures/)
- [Best Practices for Multi-Agent Systems](https://google.github.io/adk-docs/best-practices/)

---

🎉 **Congratulations!** You've mastered advanced agent architectures and know when to use each pattern!